In [3]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 4.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [4]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error as mae

from sklearn.model_selection import cross_val_score,KFold
import xgboost as xgb

from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car"


/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car


In [9]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
df =df[ df['price_currency'] != 'EUR']

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorize_value = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorize_value
  else:
     df[feat + SUFFIX_CAT] = factorize_value

In [0]:
df['param_rok-produkcji'] =df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)
df['param_moc'] =df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y,cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
feats =[
'param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_światła-led__cat']

In [12]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_reate': 0.1,
    'seed': 0
   }

run_model(xgb.XGBRegressor(**xgb_params), feats)

[17:34:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9449.513980284812, 81.47168211987172)

In [17]:
def obj_func(params):
  print('Trening with params: ')
  print(params)
  try:
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
    return{'loss': np.abs(mean_mae), 'status': STATUS_OK}
  except:
    return{'loss': np.abs(mean_mae), 'status': STATUS_FAIL}

  #space
xgb_reg_params = {
    'learning_reate':   hp.choice('learning_reate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype =int)),
    'subsample':        hp.quniform('subsample', 0.05, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree',0.05, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0
}

## run

best =  fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals = 25)

Trening with params: 
{'colsample_bytree': 0.45, 'learning_reate': 0.15000000000000002, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Trening with params: 
{'colsample_bytree': 0.2, 'learning_reate': 0.15000000000000002, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Trening with params: 
{'colsample_bytree': 0.8, 'learning_reate': 0.2, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Trening with params: 
{'colsample_bytree': 0.4, 'learning_reate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.45}
Trening with params: 
{'colsample_bytree': 0.15000000000000002, 'learning_reate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.30000000000000004}
Trening with params: 
{'colsample_by

In [18]:
best

{'colsample_bytree': 0.9,
 'learning_reate': 5,
 'max_depth': 8,
 'subsample': 0.9}